# Problem definition

> We want to predict the house prices on the test dataset.

> Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price.

# Data

* train.csv - the training set
* test.csv - the test set
* data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here
* sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms

Let's see the data description:

* SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.
* MSSubClass: The building class
* MSZoning: The general zoning classification
* LotFrontage: Linear feet of street connected to property
* LotArea: Lot size in square feet
* Street: Type of road access
* Alley: Type of alley access
* LotShape: General shape of property
* LandContour: Flatness of the property
* Utilities: Type of utilities available
* LotConfig: Lot configuration
* LandSlope: Slope of property
* Neighborhood: Physical locations within Ames city limits
* Condition1: Proximity to main road or railroad
* Condition2: Proximity to main road or railroad (if a second is present)
* BldgType: Type of dwelling
* HouseStyle: Style of dwelling
* OverallQual: Overall material and finish quality
* OverallCond: Overall condition rating
* YearBuilt: Original construction date
* YearRemodAdd: Remodel date
* RoofStyle: Type of roof
* RoofMatl: Roof material
* Exterior1st: Exterior covering on house
* Exterior2nd: Exterior covering on house (if more than one material)
* MasVnrType: Masonry veneer type
* MasVnrArea: Masonry veneer area in square feet
* ExterQual: Exterior material quality
* ExterCond: Present condition of the material on the exterior
* Foundation: Type of foundation
* BsmtQual: Height of the basement
* BsmtCond: General condition of the basement
* BsmtExposure: Walkout or garden level basement walls
* BsmtFinType1: Quality of basement finished area
* BsmtFinSF1: Type 1 finished square feet
* BsmtFinType2: Quality of second finished area (if present)
* BsmtFinSF2: Type 2 finished square feet
* BsmtUnfSF: Unfinished square feet of basement area
* TotalBsmtSF: Total square feet of basement area
* Heating: Type of heating
* HeatingQC: Heating quality and condition
* CentralAir: Central air conditioning
* Electrical: Electrical system
* 1stFlrSF: First Floor square feet
* 2ndFlrSF: Second floor square feet
* LowQualFinSF: Low quality finished square feet (all floors)
* GrLivArea: Above grade (ground) living area square feet
* BsmtFullBath: Basement full bathrooms
* BsmtHalfBath: Basement half bathrooms
* FullBath: Full bathrooms above grade
* HalfBath: Half baths above grade
* Bedroom: Number of bedrooms above basement level
* Kitchen: Number of kitchens
* KitchenQual: Kitchen quality
* TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
* Functional: Home functionality rating
* Fireplaces: Number of fireplaces
* FireplaceQu: Fireplace quality
* GarageType: Garage location
* GarageYrBlt: Year garage was built
* GarageFinish: Interior finish of the garage
* GarageCars: Size of garage in car capacity
* GarageArea: Size of garage in square feet
* GarageQual: Garage quality
* GarageCond: Garage condition
* PavedDrive: Paved driveway
* WoodDeckSF: Wood deck area in square feet
* OpenPorchSF: Open porch area in square feet
* EnclosedPorch: Enclosed porch area in square feet
* 3SsnPorch: Three season porch area in square feet
* ScreenPorch: Screen porch area in square feet
* PoolArea: Pool area in square feet
* PoolQC: Pool quality
* Fence: Fence quality
* MiscFeature: Miscellaneous feature not covered in other categories
* MiscVal: $Value of miscellaneous feature
* MoSold: Month Sold
* YrSold: Year Sold
* SaleType: Type of sale
* SaleCondition: Condition of sale

In [ ]:
!pip3 install catboost
!pip3 install xgboost
# Utilities
from xgboost import XGBRegressor
from sklearn import tree
from sklearn.linear_model import LinearRegression, RANSACRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_halving_search_cv
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, HalvingGridSearchCV
from sklearn.metrics import mean_squared_error
# Models
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor


In [ ]:
train_data = pd.read_csv('drive/MyDrive/House Prices Regression/data/train.csv')
train_data_2 = pd.read_csv('drive/MyDrive/House Prices Regression/data/AmesHousing.csv')
test_data = pd.read_csv('drive/MyDrive/House Prices Regression/data/test.csv')

In [ ]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## Data Preprocess

In [ ]:
train_data_original = train_data.copy()

In [ ]:
def show_missing_data_columns(df):
  for column in df.columns:
    if df[column].isnull().any():
      print(f'{column} {df[column].isnull().sum()/df.shape[0] * 100:.2f}% {df[column].dtypes}')

def get_missing_data_columns(df):
  column_list = []
  for column in df.columns:
    if df[column].isnull().any():
      column_list.append(column)
  return column_list

def fill_columns(df):
  '''
  Get the column with missing data.
  If string dtype, it fills with most frequent data. If numerical, it fills with median.
  '''
  for column, content in df.items():
    if df[column].isnull().any():
      if pd.api.types.is_string_dtype(content):
        print(f'Filling {column} with {df[column].value_counts().idxmax()}...')
        df[column].fillna(df[column].value_counts().idxmax(), inplace = True) # Fill with most frequent value
      else:
        print(f'Filling {column} with {df[column].median()}...')
        df[column].fillna(df[column].median(), inplace = True)

def category_converter(df):
  for label, content in df.items():
    if not pd.api.types.is_numeric_dtype(content):
        df[label] = pd.Categorical(content).codes+1 
      

In [ ]:
show_missing_data_columns(train_data)

LotFrontage 17.74% float64
Alley 93.77% object
MasVnrType 0.55% object
MasVnrArea 0.55% float64
BsmtQual 2.53% object
BsmtCond 2.53% object
BsmtExposure 2.60% object
BsmtFinType1 2.53% object
BsmtFinType2 2.60% object
Electrical 0.07% object
FireplaceQu 47.26% object
GarageType 5.55% object
GarageYrBlt 5.55% float64
GarageFinish 5.55% object
GarageQual 5.55% object
GarageCond 5.55% object
PoolQC 99.52% object
Fence 80.75% object
MiscFeature 96.30% object


In [ ]:
show_missing_data_columns(test_data)

MSZoning 0.27% object
LotFrontage 15.56% float64
Alley 92.67% object
Utilities 0.14% object
Exterior1st 0.07% object
Exterior2nd 0.07% object
MasVnrType 1.10% object
MasVnrArea 1.03% float64
BsmtQual 3.02% object
BsmtCond 3.08% object
BsmtExposure 3.02% object
BsmtFinType1 2.88% object
BsmtFinSF1 0.07% float64
BsmtFinType2 2.88% object
BsmtFinSF2 0.07% float64
BsmtUnfSF 0.07% float64
TotalBsmtSF 0.07% float64
BsmtFullBath 0.14% float64
BsmtHalfBath 0.14% float64
KitchenQual 0.07% object
Functional 0.14% object
FireplaceQu 50.03% object
GarageType 5.21% object
GarageYrBlt 5.35% float64
GarageFinish 5.35% object
GarageCars 0.07% float64
GarageArea 0.07% float64
GarageQual 5.35% object
GarageCond 5.35% object
PoolQC 99.79% object
Fence 80.12% object
MiscFeature 96.50% object
SaleType 0.07% object


In [ ]:
# Big empty data columns are no missing data, just doesn't have that feature, so let's fill with 'None' value
#no_value_columns = ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']
#for column in no_value_columns:
#  train_data[column].fillna(value = 'None', inplace = True)
#  test_data[column].fillna(value = 'None', inplace = True)


In [ ]:
show_missing_data_columns(train_data)

LotFrontage 17.74% float64
Alley 93.77% object
MasVnrType 0.55% object
MasVnrArea 0.55% float64
BsmtQual 2.53% object
BsmtCond 2.53% object
BsmtExposure 2.60% object
BsmtFinType1 2.53% object
BsmtFinType2 2.60% object
Electrical 0.07% object
FireplaceQu 47.26% object
GarageType 5.55% object
GarageYrBlt 5.55% float64
GarageFinish 5.55% object
GarageQual 5.55% object
GarageCond 5.55% object
PoolQC 99.52% object
Fence 80.75% object
MiscFeature 96.30% object


In [ ]:
# We can assume the same with all garage features (81 missing values in all cases)
garage_features = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']
for column in garage_features:
  train_data[column].fillna(value = 'None', inplace = True)
  test_data[column].fillna(value = 'None', inplace = True)

In [ ]:
show_missing_data_columns(train_data)

LotFrontage 17.74% float64
Alley 93.77% object
MasVnrType 0.55% object
MasVnrArea 0.55% float64
BsmtQual 2.53% object
BsmtCond 2.53% object
BsmtExposure 2.60% object
BsmtFinType1 2.53% object
BsmtFinType2 2.60% object
Electrical 0.07% object
FireplaceQu 47.26% object
PoolQC 99.52% object
Fence 80.75% object
MiscFeature 96.30% object


In [ ]:
# Now already we have string and numerical missing data
# Let's impute numerical with mean strategy and string with most_frequent
fill_columns(train_data)

Filling LotFrontage with 69.0...
Filling Alley with Grvl...
Filling MasVnrType with None...
Filling MasVnrArea with 0.0...
Filling BsmtQual with TA...
Filling BsmtCond with TA...
Filling BsmtExposure with No...
Filling BsmtFinType1 with Unf...
Filling BsmtFinType2 with Unf...
Filling Electrical with SBrkr...
Filling FireplaceQu with Gd...
Filling PoolQC with Gd...
Filling Fence with MnPrv...
Filling MiscFeature with Shed...


In [ ]:
fill_columns(test_data)

Filling MSZoning with RL...
Filling LotFrontage with 67.0...
Filling Alley with Grvl...
Filling Utilities with AllPub...
Filling Exterior1st with VinylSd...
Filling Exterior2nd with VinylSd...
Filling MasVnrType with None...
Filling MasVnrArea with 0.0...
Filling BsmtQual with TA...
Filling BsmtCond with TA...
Filling BsmtExposure with No...
Filling BsmtFinType1 with GLQ...
Filling BsmtFinSF1 with 350.5...
Filling BsmtFinType2 with Unf...
Filling BsmtFinSF2 with 0.0...
Filling BsmtUnfSF with 460.0...
Filling TotalBsmtSF with 988.0...
Filling BsmtFullBath with 0.0...
Filling BsmtHalfBath with 0.0...
Filling KitchenQual with TA...
Filling Functional with Typ...
Filling FireplaceQu with Gd...
Filling GarageCars with 2.0...
Filling GarageArea with 480.0...
Filling PoolQC with Ex...
Filling Fence with MnPrv...
Filling MiscFeature with Shed...
Filling SaleType with WD...


In [ ]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Grvl,Reg,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Grvl,Reg,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,Grvl,IR1,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,Grvl,IR1,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,Grvl,IR1,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,12,2008,WD,Normal,250000


In [ ]:
# Now convert the string to category codes
category_converter(train_data)
category_converter(test_data)

In [ ]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,4,65.0,8450,2,1,4,4,1,...,0,3,3,3,0,2,2008,9,5,208500
1,2,20,4,80.0,9600,2,1,4,4,1,...,0,3,3,3,0,5,2007,9,5,181500
2,3,60,4,68.0,11250,2,1,1,4,1,...,0,3,3,3,0,9,2008,9,5,223500
3,4,70,4,60.0,9550,2,1,1,4,1,...,0,3,3,3,0,2,2006,9,1,140000
4,5,60,4,84.0,14260,2,1,1,4,1,...,0,3,3,3,0,12,2008,9,5,250000


## Data Analysis

In [ ]:
# Mutual Information
X = train_data.drop(labels = 'SalePrice', axis = 1)
y = train_data['SalePrice']
mi_score = mutual_info_regression(X, y)
mi_df = pd.DataFrame({'Feature': X.columns,
                      'MI_score': mi_score}).sort_values(by = ['MI_score'], ascending = False)
mi_df['MI_score'].astype(float)

mi_df.head(20)

,Feature,MI_score
17,OverallQual,0.550904
12,Neighborhood,0.509604
46,GrLivArea,0.480661
61,GarageCars,0.385189
38,TotalBsmtSF,0.366956
62,GarageArea,0.364389
19,YearBuilt,0.361291
53,KitchenQual,0.326003
43,1stFlrSF,0.310867
30,BsmtQual,0.307508


In [ ]:
mi_df.tail(20)

,Feature,MI_score
21,RoofStyle,0.013958
48,BsmtHalfBath,0.012459
52,KitchenAbvGr,0.011876
72,PoolQC,0.010569
39,Heating,0.009795
77,YrSold,0.009761
45,LowQualFinSF,0.008140
22,RoofMatl,0.007618
71,PoolArea,0.005573
36,BsmtFinSF2,0.004768


In [ ]:
useless_features = [column for column in mi_df[mi_df['MI_score'] < 0.06]['Feature']]
useless_features

['GarageQual',
 'Electrical',
 'BldgType',
 'PavedDrive',
 'BsmtCond',
 'EnclosedPorch',
 'Alley',
 'BsmtFullBath',
 'LotConfig',
 'ExterCond',
 'ScreenPorch',
 'LandContour',
 'Condition1',
 'Condition2',
 'Fence',
 'RoofStyle',
 'BsmtHalfBath',
 'KitchenAbvGr',
 'PoolQC',
 'Heating',
 'YrSold',
 'LowQualFinSF',
 'RoofMatl',
 'PoolArea',
 'BsmtFinSF2',
 'LandSlope',
 'BsmtFinType2',
 'Functional',
 '3SsnPorch',
 'Utilities',
 'MiscFeature',
 'MiscVal',
 'MoSold',
 'Street',
 'Id']

In [ ]:
for column in useless_features:
  if column != 'Id':
    train_data.drop(labels = column, axis = 1, inplace = True)
    test_data.drop(labels = column, axis = 1, inplace = True)

print(len(train_data.columns))

47


# Modelling

## First Model

In [ ]:
# Let's split the data into train and validations sets
#np.random.seed(21)
train_set, val_set = train_test_split(train_data,
                                      test_size = 0.2,
                                      shuffle = True)

train_set.shape, val_set.shape

((1168, 47), (292, 47))

In [ ]:
# Now we're going to split both sets into X and y
X_train, y_train = train_set.drop(labels = 'SalePrice', axis = 1), train_set['SalePrice']
X_val, y_val = val_set.drop(labels = 'SalePrice', axis = 1), val_set['SalePrice']

In [ ]:
# We are going to try first with CatBoostRegressor and XGBoost
#np.random.seed(21)

cbr = CatBoostRegressor(verbose = False)
xgbr = XGBRegressor(verbosity = 0)
rfr = RandomForestRegressor()
lr = LinearRegression()
treer = tree.DecisionTreeRegressor()

cbr.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
rfr.fit(X_train, y_train)
lr.fit(X_train, y_train)
treer.fit(X_train, y_train)

print(f'CatBoostRegressor: {cbr.score(X_val, y_val)}')
print(f'XGBRegressor: {xgbr.score(X_val, y_val)}')
print(f'RandomForestRegressor: {rfr.score(X_val, y_val)}')
print(f'LinearRegression: {lr.score(X_val, y_val)}')

CatBoostRegressor: 0.9103464973738118
XGBRegressor: 0.8618725594516938
RandomForestRegressor: 0.8988127184352843
LinearRegression: 0.8472712959935818


In [ ]:
y_val_preds = cbr.predict(X_val)
np.sqrt(mean_squared_error(y_val, y_val_preds))

26740.25191884935

In [ ]:
y_preds = cbr.predict(test_data)

In [ ]:
house_price_predictions = pd.DataFrame({'Id': test_data['Id'],
                                        'SalePrice': y_preds})
house_price_predictions

,Id,SalePrice
0,1461,125481.776820
1,1462,167096.004432
2,1463,185544.087777
3,1464,192750.147977
4,1465,183042.891144
...,...,...
1454,2915,81776.557691
1455,2916,82180.480707
1456,2917,165849.355264
1457,2918,114470.270649


In [ ]:
house_price_predictions.to_csv('drive/MyDrive/House Prices Regression/predictions/house_price_predictions.csv',
                               index = False)

**We can see that CatBoostRegressor works better than any of the other models 0.8549701588350334 r2**

## Model Tuning

In [ ]:
cbr.get_all_params()

{'auto_class_weights': 'None',
 'bayesian_matrix_reg': 0.10000000149011612,
 'best_model_min_trees': 1,
 'boost_from_average': True,
 'boosting_type': 'Plain',
 'bootstrap_type': 'MVS',
 'border_count': 254,
 'classes_count': 0,
 'depth': 6,
 'eval_metric': 'RMSE',
 'feature_border_type': 'GreedyLogSum',
 'force_unit_auto_pair_weights': False,
 'grow_policy': 'SymmetricTree',
 'iterations': 1000,
 'l2_leaf_reg': 3,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'leaf_estimation_iterations': 1,
 'leaf_estimation_method': 'Newton',
 'learning_rate': 0.04196000099182129,
 'loss_function': 'RMSE',
 'max_leaves': 64,
 'min_data_in_leaf': 1,
 'model_shrink_mode': 'Constant',
 'model_shrink_rate': 0,
 'model_size_reg': 0.5,
 'nan_mode': 'Min',
 'penalties_coefficient': 1,
 'pool_metainfo_options': {'tags': {}},
 'posterior_sampling': False,
 'random_seed': 0,
 'random_strength': 1,
 'rsm': 1,
 'sampling_frequency': 'PerTree',
 'score_function': 'Cosine',
 'sparse_features_conflict_fracti

In [ ]:
# Let's tune CatBoostRegressor
#param_grid = {
#    'iterations': np.arange(100, 1000, 200),
#    'learning_rate': np.arange(0.01, 0.9, 0.5),
#    'depth': np.arange(4, 10, 1)
#}
#
#cbr_hs = HalvingGridSearchCV(estimator = cbr,
#                             param_grid = param_grid,
#                             cv = 5,
#                             factor = 3,
#                             n_jobs = -1,
#                             verbose = True)
#
#cbr_hs.fit(X_train, y_train)

In [ ]:
cbr_hs = CatBoostRegressor(depth = 5, iterations = 900, learning_rate = 0.01, verbose = False)
cbr_hs.fit(X_train, y_train)
cbr_hs.score(X_val, y_val)

0.8891260465956855

# Other Data

There's another dataset from Kaggle we can use for improve our model! 😄

In [ ]:
train_data_2.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [ ]:
len(train_data_original.columns), len(train_data_2.columns)

(81, 82)

## Data Preprocess

In [ ]:
def unique_columns_classifier(df_1, df_2):
  classifier = []
  df_1_columns = [column.replace(' ', '') for column in df_1.columns]
  df_2_columns = [column.replace(' ', '') for column in df_2.columns]
  unique_columns = np.unique(df_1_columns + df_2_columns)
  for column in df_2_columns:
    if column not in df_1_columns:
      print(f'Column {column} from df_2 non in df_1')
  
  for column in df_1_columns:
    if column not in df_2_columns:
      print(f'Column {column} from df_1 non in df_2')

In [ ]:
unique_columns_classifier(train_data_original, train_data_2)

Column Order from df_2 non in df_1
Column PID from df_2 non in df_1
Column YearRemod/Add from df_2 non in df_1
Column Id from df_1 non in df_2
Column YearRemodAdd from df_1 non in df_2


In [ ]:
train_data_2.rename(columns = {'PID':'Id', 'Year Remod/Add':'YearRemodAdd'}, inplace = True)
train_data_2.drop(labels = 'Order', axis = 1, inplace = True)

In [ ]:
train_data_2.columns = train_data_2.columns.str.replace(' ','')

In [ ]:
garage_features = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']
for column in garage_features:
  train_data_2[column].fillna(value = 'None', inplace = True)

In [ ]:
fill_columns(train_data_2)
category_converter(train_data_2)

Filling LotFrontage with 68.0...
Filling Alley with Grvl...
Filling MasVnrType with None...
Filling MasVnrArea with 0.0...
Filling BsmtQual with TA...
Filling BsmtCond with TA...
Filling BsmtExposure with No...
Filling BsmtFinType1 with GLQ...
Filling BsmtFinSF1 with 370.0...
Filling BsmtFinType2 with Unf...
Filling BsmtFinSF2 with 0.0...
Filling BsmtUnfSF with 466.0...
Filling TotalBsmtSF with 990.0...
Filling Electrical with SBrkr...
Filling BsmtFullBath with 0.0...
Filling BsmtHalfBath with 0.0...
Filling FireplaceQu with Gd...
Filling GarageCars with 2.0...
Filling GarageArea with 480.0...
Filling PoolQC with Ex...
Filling Fence with MnPrv...
Filling MiscFeature with Shed...


In [ ]:
for column in useless_features:
  if column != 'Id':
    train_data_2.drop(labels = column, axis = 1, inplace = True)
print(len(train_data_2.columns))

47


In [ ]:
len(train_data.columns), len(train_data_2.columns)

(47, 47)

In [ ]:
train_data_full = pd.concat([train_data, train_data_2])

In [ ]:
train_data_full

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,Neighborhood,HouseStyle,OverallQual,OverallCond,...,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageCond,WoodDeckSF,OpenPorchSF,SaleType,SaleCondition,SalePrice
0,1,60,4,65.0,8450,4,6,6,7,5,...,90,3,2.0,548.0,6,0,61,9,5,208500
1,2,20,4,80.0,9600,4,25,3,6,8,...,63,3,2.0,460.0,6,298,0,9,5,181500
2,3,60,4,68.0,11250,1,6,6,7,5,...,88,3,2.0,608.0,6,0,42,9,5,223500
3,4,70,4,60.0,9550,1,7,6,7,5,...,85,4,3.0,642.0,6,0,35,9,1,140000
4,5,60,4,84.0,14260,1,16,6,8,5,...,87,3,3.0,836.0,6,192,84,9,5,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,923275080,80,6,37.0,7937,1,15,8,6,6,...,76,4,2.0,588.0,6,120,0,10,5,142500
2926,923276100,20,6,68.0,8885,1,15,3,5,5,...,75,4,2.0,484.0,6,164,0,10,5,131000
2927,923400125,85,6,62.0,10441,4,15,7,5,5,...,104,2,0.0,0.0,4,80,32,10,5,132000
2928,924100070,20,6,77.0,10010,4,15,3,5,5,...,67,3,2.0,418.0,6,240,38,10,5,170000


## Data Improved Train Model

In [ ]:
train_set, val_set = train_test_split(train_data_full,
                                      test_size = 0.2,
                                      shuffle = True)

train_set.shape, val_set.shape

((3512, 47), (878, 47))

In [ ]:
X_train, y_train = train_set.drop(labels = 'SalePrice', axis = 1), train_set['SalePrice']
X_val, y_val = val_set.drop(labels = 'SalePrice', axis = 1), val_set['SalePrice']

In [ ]:
cbr = CatBoostRegressor(verbose = False)
xgbr = XGBRegressor(verbosity = 0)
rfr = RandomForestRegressor()
lr = LinearRegression()
treer = tree.DecisionTreeRegressor()

cbr.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
rfr.fit(X_train, y_train)
lr.fit(X_train, y_train)
treer.fit(X_train, y_train)

print(f'CatBoostRegressor: {cbr.score(X_val, y_val)}')
print(f'XGBRegressor: {xgbr.score(X_val, y_val)}')
print(f'RandomForestRegressor: {rfr.score(X_val, y_val)}')
print(f'LinearRegression: {lr.score(X_val, y_val)}')

CatBoostRegressor: 0.958138119712045
XGBRegressor: 0.9321386576380903
RandomForestRegressor: 0.9493910741340821
LinearRegression: 0.8514692305702429


In [ ]:
test_data = test_data.reindex(sorted(test_data.columns), axis=1)
train_data_full = train_data_full.reindex(sorted(train_data_full.columns), axis=1)

In [ ]:
test_data.columns, train_data_full.columns

(Index(['1stFlrSF', '2ndFlrSF', 'BedroomAbvGr', 'BsmtExposure', 'BsmtFinSF1',
        'BsmtFinType1', 'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'ExterQual',
        'Exterior1st', 'Exterior2nd', 'FireplaceQu', 'Fireplaces', 'Foundation',
        'FullBath', 'GarageArea', 'GarageCars', 'GarageCond', 'GarageFinish',
        'GarageType', 'GarageYrBlt', 'GrLivArea', 'HalfBath', 'HeatingQC',
        'HouseStyle', 'Id', 'KitchenQual', 'LotArea', 'LotFrontage', 'LotShape',
        'MSSubClass', 'MSZoning', 'MasVnrArea', 'MasVnrType', 'Neighborhood',
        'OpenPorchSF', 'OverallCond', 'OverallQual', 'SaleCondition',
        'SaleType', 'TotRmsAbvGrd', 'TotalBsmtSF', 'WoodDeckSF', 'YearBuilt',
        'YearRemodAdd'],
       dtype='object'),
 Index(['1stFlrSF', '2ndFlrSF', 'BedroomAbvGr', 'BsmtExposure', 'BsmtFinSF1',
        'BsmtFinType1', 'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'ExterQual',
        'Exterior1st', 'Exterior2nd', 'FireplaceQu', 'Fireplaces', 'Foundation',
        'FullBath', '

In [ ]:
y_preds_improved = cbr.predict(test_data)

In [ ]:
house_price_predictions_improved = pd.DataFrame({'Id': test_data['Id'],
                                        'SalePrice': y_preds_improved})
house_price_predictions_improved

,Id,SalePrice
0,1461,114870.497018
1,1462,170287.518214
2,1463,186638.771725
3,1464,199981.571492
4,1465,176979.611307
...,...,...
1454,2915,85432.104881
1455,2916,81147.428139
1456,2917,139943.696042
1457,2918,125256.185930


In [ ]:
house_price_predictions_improved.to_csv('drive/MyDrive/House Prices Regression/predictions/house_price_predictions_improved.csv',
                               index = False)